In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
import neptune.new as neptune
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import IPython.display as ipd
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger

In [2]:
load_dotenv()
neptune_logger = NeptuneLogger(
    project="pavle/mlav",
    api_key=os.getenv("NEPTUNE_TOKEN"),
    log_model_checkpoints=False,
)

In [3]:
labels = ['backward', 'down', 'forward', 'go', 'left', 'off', 'on', 'right', 'stop', 'up']

In [4]:
class SpeechCommands(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=False)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validate":
            self._walker = load_list("validation_list_cleaned.txt")
        elif subset == "test":
            self._walker = load_list("testing_list_cleaned.txt")
        elif subset == "train":
            excludes = load_list("validation_list_cleaned.txt") + load_list("testing_list_cleaned.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

In [5]:
def label_to_index(word):
    return torch.tensor(labels.index(word))

def index_to_label(index):
    return labels[index]

In [6]:
class SpeechDataModule(LightningDataModule):
    def __init__(self, batch_size, labels=labels, transform=None):
        super().__init__()
        self.batch_size = batch_size
        self.labels = labels
        self.transform = transform

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_set = SpeechCommands("train")
            self.valid_set = SpeechCommands("validate")

        if stage == "test" or stage is None:
            self.test_set = SpeechCommands("test")

    def train_dataloader(self):
        loader = DataLoader(self.train_set,
                            batch_size=self.batch_size,
                            shuffle=True,
                            drop_last=True,
                            collate_fn=self.collate_fn,
                            pin_memory=True,
                            num_workers=4
                           )
        return loader

    def val_dataloader(self):
        loader = DataLoader(self.valid_set,
                            batch_size=self.batch_size,
                            shuffle=False,
                            drop_last=False,
                            collate_fn=self.collate_fn,
                            pin_memory=True,
                            num_workers=4
                           )
        return loader

    def test_dataloader(self):
        loader = DataLoader(self.test_set,
                            batch_size=self.batch_size,
                            shuffle=False,
                            drop_last=False,
                            collate_fn=self.collate_fn,
                            pin_memory=True,
                            num_workers=4
                           )
        return loader

    def pad_sequence(self, batch):
        batch = [item.t() for item in batch]
        batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
        batch = self.transform(batch.permute(0, 2, 1))
        return batch

    def label_to_index(self, word):
        return torch.tensor(self.labels.index(word))

    def collate_fn(self, batch):
        tensors, targets = [], []

        for waveform, _, label, _, _ in batch:
            tensors += [waveform]
            targets += [self.label_to_index(label)]

        tensors = self.pad_sequence(tensors)
        targets = torch.stack(targets)

        return tensors, targets

In [7]:
train_set = SpeechCommands("train")
test_set = SpeechCommands("test")

In [8]:
class VoiceRecognizerNN(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)

In [9]:
class VoiceRecognizer(LightningModule):
    def __init__(self, model_hparams, optimizer_hparams):
        super().__init__()
        self.optimizer_hparams = optimizer_hparams
        self.model = VoiceRecognizerNN(model_hparams["n_input"],
                        model_hparams["n_output"],
                        model_hparams["stride"],
                        model_hparams["n_channel"]
                       )

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=2)

    def training_step(self, batch, batch_idx):
        data, target = batch
        output = self(data).squeeze()
        loss = F.nll_loss(output, target)
        self.log("tr_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, target = batch
        output = self(data).squeeze()
        loss = F.nll_loss(output, target)

        preds = torch.argmax(output, dim=1)
        acc = (target == preds).float().mean()

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        data, target = batch
        output = self(data).squeeze()

        preds = torch.argmax(output, dim=1)
        acc = (target == preds).float().mean()
        self.log("test_acc", acc, prog_bar=True)
        return acc

    def configure_optimizers(self):
        params = self.optimizer_hparams
        optimizer = torch.optim.AdamW(self.parameters(),
                                      lr=params["learning_rate"],
                                      weight_decay=params["weight_decay"]
                                     )
        return optimizer

In [10]:
batch_size = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

cuda


In [11]:
epochs = 50
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath='./speech/models/', filename='spcm_{epoch:03d}', save_top_k=5)
model_hparams = {
    "n_input": 1,
    "n_output": 10,
    "stride": 16,
    "n_channel": 32,
}

optimizer_hparams = {
    "learning_rate": 3e-3,
    "weight_decay": 0,
    "epochs": epochs,
    "batch_size": batch_size
}
neptune_logger.log_hyperparams(params=optimizer_hparams)
model = VoiceRecognizer(model_hparams, optimizer_hparams)

https://app.neptune.ai/pavle/mlav/e/MLAV-10
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [12]:
trainer = Trainer(
    max_epochs=epochs,
    logger=neptune_logger,
    accelerator="gpu" if str(device).startswith("cuda") else "cpu",
    precision=16,
    callbacks=[checkpoint_callback]
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
orig_freq = 16000
new_freq = 8000
transform = torchaudio.transforms.Resample(orig_freq=orig_freq, new_freq=new_freq)

In [14]:
dm = SpeechDataModule(batch_size=batch_size, transform=transform)

In [15]:
trainer.fit(model, dm)

/home/pavle/Documents/MLAV/venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /home/pavle/Documents/MLAV/voice_recognizer/speech/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | model | VoiceRecognizerNN | 25.3 K
--------------------------------------------
25.3 K    Trainable params
0         Non-trainable params
25.3 K    Total params
0.051     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [16]:
trainer.test(model, dm)
trainer.logger.experiment.stop()


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8579831719398499
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 16 operations to synchronize with Neptune. Do not kill this process.
All 16 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/pavle/mlav/e/MLAV-10


In [20]:
m = torch.jit.script(model)
torch.jit.save(m, "voice_recognizer.pt")

In [17]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 3
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Could not import the PyAudio C module 'pyaudio._portaudio'.


ImportError: libportaudio.so.2: cannot open shared object file: No such file or directory